In [15]:
# %pip install opencv-python deepface mysql-connector-python numpy scipy ulid-py requests

In [16]:
import cv2
from deepface import DeepFace
import mysql.connector
import os
import time
from datetime import datetime
import socket
import numpy as np
import json
from scipy.spatial.distance import cosine
import ulid  # For ULID face IDs
import requests # added to send images to API
import time

In [17]:
def detect_faces_in_frame(frame, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60)):
    """Return list of (x,y,w,h) for detected faces in BGR frame using OpenCV Haar cascade."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize)
    return faces

def should_capture_frame(frame, min_face_area=2000):
    """Return (bool, faces). True only if at least one detected face is larger than min_face_area."""
    faces = detect_faces_in_frame(frame)
    if len(faces) == 0:
        return False, []
    big_faces = [f for f in faces if f[2] * f[3] >= min_face_area]
    return (len(big_faces) > 0), big_faces

# Usage example (call this in your capture loop BEFORE saving/sending):
# ret, frame = cap.read()
# should_capture, faces = should_capture_frame(frame)
# if should_capture:
#     # crop first face (or iterate faces) and encode/send
#     x, y, w, h = faces[0]
#     face_img = frame[y:y+h, x:x+w]
#     ret, buf = cv2.imencode('.jpg', face_img)
#     if ret:
#         img_bytes = buf.tobytes()
#         ok, body = send_image_to_api_safe(img_bytes, pc_name=socket.gethostname())
#         # handle ok/body as needed
# else:
#     # no face -> skip saving/sending
#     pass

# ==== Configuration ====


In [18]:
CHECK_INTERVAL = 10
MAX_CAMERA_USE_TIME = 10  # seconds camera will be used before release
CAPTURE_PAUSE = 30        # seconds to wait after releasing camera
CAPTURED_FACES_DIR = "captured_faces"
EMBEDDING_MODEL = "ArcFace"
MATCH_THRESHOLD = 0.45  # Adjust as needed
API_URL = "http://127.0.0.1:8000/upload-face"  # endpoint to send captured images
CLIENT_API_KEY = os.getenv('API_KEY', 'replace-me-with-a-secure-key')  # used for X-API-Key header

# ==== Ensure storage directory exists ====


In [19]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# ==== Get Embeddings DB ====

In [20]:
def get_embeddings_db(cursor):
    cursor.execute("SELECT face_id, embedding FROM captured_snapshots WHERE embedding IS NOT NULL")
    known = []
    for face_id, emb_json in cursor.fetchall():
        try:
            emb = np.array(json.loads(emb_json), dtype=np.float64)
            known.append((face_id, emb))
        except Exception:
            continue
    return known

# ==== Get Face Embedding ====

In [21]:
def get_face_embedding(face_img):
    try:
        # DeepFace expects a file path or numpy array (BGR)
        reps = DeepFace.represent(face_img, model_name=EMBEDDING_MODEL, enforce_detection=False)
        if reps and isinstance(reps, list):
            emb = reps[0]['embedding']
            return np.array(emb, dtype=np.float64)
    except Exception as e:
        print(f"❌ Embedding error: {e}")
    return None

# ==== Match Face ID ====

In [22]:
def match_face_id(embedding, known, threshold=MATCH_THRESHOLD):
    for face_id, known_emb in known:
        dist = cosine(embedding, known_emb)
        if dist < threshold:
            return face_id
    return None

# ==== Generate ULID Face ID ====

In [23]:
# ==== Generate ULID Face ID ====
def generate_ulid():
    return str(ulid.new())

# ==== Assign and Reuse Face IDs ====

In [24]:
# def get_or_create_face_id(pc_name):
#     face_id_file = f"{pc_name}_face_id.txt"
#     if os.path.exists(face_id_file):
#         with open(face_id_file, "r") as f:
#             return f.read().strip()
#     else:
#         import uuid
#         face_id = f"{pc_name}_{uuid.uuid4().hex[:8]}"
#         with open(face_id_file, "w") as f:
#             f.write(face_id)
#         return face_id

# ====  Save to Database ====

In [25]:
def save_snapshot_to_db(self, face_id, pc_name, image_path, timestamp, embedding):
    sql = """
        INSERT INTO captured_snapshots (face_id, pc_name, image_path, timestamp, embedding)
        VALUES (%s, %s, %s, %s, %s)
    """
    emb_json = json.dumps(embedding.tolist()) if embedding is not None else None

    # Ensure timestamp is a datetime object
    if isinstance(timestamp, str):
        timestamp = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')

    self.cursor.execute(sql, (face_id, pc_name, image_path, timestamp, emb_json))
    self.db.commit()


In [26]:
# Helper: send image bytes to the API endpoint (robust)
import requests
import time
from urllib.parse import urlparse

def send_image_to_api_safe(image_bytes, api_url=API_URL, pc_name=None, timeout=10, retries=2, backoff=1.0, headers=None):
    """Send a JPEG byte buffer to the API as multipart/form-data.
    Returns a tuple (ok:bool, body: dict|str).
    Logs every response to 'api_responses.log' with a timestamp.
    This helper does not raise on network errors; it returns (False, error_message).
    """
    if pc_name is None:
        pc_name = socket.gethostname()
    if headers is None:
        headers = {'X-API-Key': CLIENT_API_KEY}

    files = {
        'file': ('capture.jpg', image_bytes, 'image/jpeg')
    }
    data = {'pc_name': pc_name}

    last_exception = None
    for attempt in range(retries + 1):
        try:
            resp = requests.post(api_url, files=files, data=data, headers=headers, timeout=timeout)
            status = resp.status_code
            try:
                body = resp.json()
            except Exception:
                body = resp.text

            # Log response for auditing
            try:
                log_entry = {
                    'timestamp': datetime.utcnow().isoformat() + 'Z',
                    'api_url': api_url,
                    'status_code': status,
                    'body': body
                }
                with open('api_responses.log', 'a', encoding='utf-8') as f:
                    f.write(json.dumps(log_entry, ensure_ascii=False) + '\n')
            except Exception as log_err:
                print(f"⚠️ Failed to write API log: {log_err}")

            return (status == 200, body)

        except requests.exceptions.RequestException as e:
            last_exception = e
            # simple backoff then retry
            time.sleep(backoff * (attempt + 1))
            continue

    # all retries failed — return a clear failure tuple instead of raising
    err_msg = str(last_exception) if last_exception is not None else 'Unknown network error'
    try:
        with open('api_responses.log', 'a', encoding='utf-8') as f:
            f.write(json.dumps({'timestamp': datetime.utcnow().isoformat() + 'Z', 'error': err_msg}) + '\n')
    except Exception:
        pass
    return (False, err_msg)

# Small helper to check API health before starting background capture
def check_api_health(api_url=API_URL, timeout=2):
    """Check the server health endpoint at scheme://host:port/health regardless of the API endpoint path."""
    try:
        parsed = urlparse(api_url)
        if parsed.scheme and parsed.netloc:
            health_url = f"{parsed.scheme}://{parsed.netloc}/health"
        else:
            # fallback (should not generally happen) — keep previous behavior
            health_url = api_url.rstrip('/') + '/health'
        r = requests.get(health_url, timeout=timeout)
        return r.status_code == 200
    except Exception:
        return False


# ==== Main Background Face Recognition Class ====


In [27]:
class BackgroundFaceCapture:
    def __init__(self):
        print("Initializing background face capture...")
        ensure_dir(CAPTURED_FACES_DIR)
        self.db = None  # keep None unless DB integration is added
        self.running = True

    def _save_local(self, img_bytes):
        """Save image bytes to local queue directory for later upload."""
        try:
            ts = datetime.utcnow().strftime('%Y%m%dT%H%M%S_%f')
            fname = f"capture_{socket.gethostname()}_{ts}.jpg"
            path = os.path.join(CAPTURED_FACES_DIR, fname)
            with open(path, 'wb') as f:
                f.write(img_bytes)
            return path
        except Exception as e:
            print(f"⚠️ Failed to save local capture: {e}")
            return None

    def _flush_local_queue(self):
        """Attempt to send any locally stored images to the API. Remove on success."""
        if not os.path.isdir(CAPTURED_FACES_DIR):
            return
        files = sorted(os.listdir(CAPTURED_FACES_DIR))
        for f in files:
            full = os.path.join(CAPTURED_FACES_DIR, f)
            # small safety: only process .jpg files we wrote
            if not f.lower().endswith('.jpg'):
                continue
            try:
                with open(full, 'rb') as fh:
                    data = fh.read()
                ok, body = send_image_to_api_safe(data, api_url=API_URL, pc_name=socket.gethostname())
                if ok:
                    try:
                        os.remove(full)
                    except Exception:
                        pass
                else:
                    # stop trying further files if API rejects to avoid tight loop
                    break
            except Exception as e:
                print(f"⚠️ Error flushing {full}: {e}")
                continue

    def _ensure_flushed(self, max_wait=60, interval=2):
        """Keep retrying to send all queued images until directory is empty or max_wait seconds elapsed.
        Returns True if queue emptied, False otherwise."""
        start = time.time()
        while True:
            # quick exit if no files
            if not os.path.isdir(CAPTURED_FACES_DIR):
                return True
            files = [f for f in sorted(os.listdir(CAPTURED_FACES_DIR)) if f.lower().endswith('.jpg')]
            if not files:
                return True

            # attempt to send each file; if any send fails, we'll retry after a short sleep
            any_failure = False
            for f in files:
                full = os.path.join(CAPTURED_FACES_DIR, f)
                try:
                    with open(full, 'rb') as fh:
                        data = fh.read()
                    ok, body = send_image_to_api_safe(data, api_url=API_URL, pc_name=socket.gethostname())
                    if ok:
                        try:
                            os.remove(full)
                        except Exception:
                            pass
                    else:
                        any_failure = True
                        print(f"⚠️ Flush failed for {f}: {body}")
                        # continue to next file to attempt best-effort; overall we'll retry later
                except Exception as e:
                    any_failure = True
                    print(f"⚠️ Exception flushing {full}: {e}")

            if not any_failure:
                # everything sent
                return True

            # stop if waited too long
            if (time.time() - start) >= max_wait:
                return False

            # wait then retry
            time.sleep(interval)

    def _process_frame(self, frame):
        """Return JPEG bytes for the first qualifying face in the frame or None."""
        should_capture, faces = should_capture_frame(frame)
        if not should_capture or not faces:
            return None
        x, y, w, h = faces[0]
        face_img = frame[y:y+h, x:x+w]
        ret, buf = cv2.imencode('.jpg', face_img)
        if not ret:
            return None
        return buf.tobytes()

    def run(self):
        print("Running background face capture service...")
        while self.running:
            # Attempt to open the camera (handle camera-in-use by other apps)
            cap = None
            try_count = 0
            while True:
                try_count += 1
                cap = cv2.VideoCapture(0)
                if cap is not None and cap.isOpened():
                    break
                # Camera not available; release handle and wait before retrying
                if cap is not None:
                    try:
                        cap.release()
                    except Exception:
                        pass
                print(f"Camera unavailable (attempt {try_count}). Waiting {CHECK_INTERVAL}s...")
                time.sleep(CHECK_INTERVAL)

            # Camera opened successfully; capture for configured duration
            end_time = time.time() + MAX_CAMERA_USE_TIME
            print(f"Camera opened — capturing for {MAX_CAMERA_USE_TIME}s...")
            while time.time() < end_time:
                ret, frame = cap.read()
                if not ret or frame is None:
                    # Could be the camera was taken over; break to outer loop to re-acquire
                    print("⚠️ Failed to read from camera; breaking capture loop.")
                    break

                img_bytes = self._process_frame(frame)
                if img_bytes is None:
                    # nothing to send this frame
                    time.sleep(0.25)
                    continue

                # Try sending immediately to the API. If send succeeds, attempt to flush the local queue.
                # If send fails, persist locally for later retry.
                try:
                    ok, body = send_image_to_api_safe(img_bytes, api_url=API_URL, pc_name=socket.gethostname())
                except Exception as e:
                    ok, body = False, str(e)

                if ok:
                    # best-effort small flush after a successful send
                    try:
                        self._ensure_flushed(max_wait=10, interval=1)
                    except Exception as e:
                        print(f"⚠️ Background flush failed: {e}")
                else:
                    # save locally for later retry
                    print(f"⚠️ Send failed — saving locally: {body}")
                    self._save_local(img_bytes)

                # small backoff to avoid overloading CPU
                time.sleep(0.25)

            # Release camera and pause before trying again
            try:
                cap.release()
            except Exception:
                pass
            print(f"Released camera. Pausing for {CAPTURE_PAUSE}s before next activation.")

            # During pause, avoid excessive health-checking: attempt a periodic flush only (every 15s)
            pause_until = time.time() + CAPTURE_PAUSE
            next_flush = time.time() + 15  # first periodic flush after 15s
            while True:
                remaining = pause_until - time.time()
                if remaining <= 0:
                    break

                now = time.time()
                if now >= next_flush:
                    # Only attempt to flush queued files periodically
                    try:
                        # non-blocking short flush during pause
                        self._ensure_flushed(max_wait=8, interval=1)
                    except Exception as e:
                        print(f"⚠️ Error flushing during pause: {e}")
                    next_flush = now + 15

                # sleep a short amount (bounded by remaining) to avoid tight loop
                time.sleep(min(2, max(0.1, remaining)))

        print("BackgroundFaceCapture stopping")

In [28]:
# Use this check before starting the service; it prints clear status for the user.
if __name__ == '__main__':
    available = check_api_health(API_URL)
    if available:
        print(f"✅ API appears reachable at {API_URL} — captured images will be uploaded.")
    else:
        print(f"⚠️ API not reachable at {API_URL}. Images will be saved locally and retried until the API is available.")

    recognizer = None
    try:
        recognizer = BackgroundFaceCapture()
        recognizer.run()
    except KeyboardInterrupt:
        print("🛑 Background service stopped manually.")
    except Exception as e:
        print(f"❌ Unexpected error in service: {e}")
    finally:
        # Try to flush any queued images before exit if API is reachable
        try:
            if recognizer is not None and check_api_health(API_URL):
                print("Attempting to flush queued images before exit...")
                try:
                    recognizer._ensure_flushed(max_wait=60, interval=2)
                except Exception as e:
                    print(f"⚠️ Error while flushing on shutdown: {e}")
        except Exception:
            pass

        # Close DB if present
        try:
            if recognizer is not None and getattr(recognizer, 'db', None):
                recognizer.db.close()
        except Exception:
            pass

        print('Shutdown complete.')

✅ API appears reachable at http://127.0.0.1:8000/upload-face — captured images will be uploaded.
Initializing background face capture...
Running background face capture service...
Camera opened — capturing for 10s...
Camera opened — capturing for 10s...
Released camera. Pausing for 30s before next activation.
Released camera. Pausing for 30s before next activation.
Camera opened — capturing for 10s...
Camera opened — capturing for 10s...
Released camera. Pausing for 30s before next activation.
Released camera. Pausing for 30s before next activation.
Camera opened — capturing for 10s...
Camera opened — capturing for 10s...
Released camera. Pausing for 30s before next activation.
Released camera. Pausing for 30s before next activation.
Camera opened — capturing for 10s...
Camera opened — capturing for 10s...
Released camera. Pausing for 30s before next activation.
Released camera. Pausing for 30s before next activation.
Camera opened — capturing for 10s...
Camera opened — capturing for 1